In [1]:
import matplotlib.pyplot as plt
from collections import namedtuple
import pprint
#from ase.build import molecule
#from weas_widget import WeasWidget
#from matplotlib.ticker import ScalarFormatter
import glob
#from collections import defaultdict
from pathlib import Path
import numpy as np
import csv
import pandas as pd
pd.set_option('display.width', 1000)
pd.set_option('display.float_format', '{:e}'.format)
from math import isclose
#from lmfit.models import QuadraticModel, LorentzianModel

### List the named calculation directory you created with Full_Polarization.py. This is the point of entry for all data analysis.

In [2]:
parent_dir = "/home/sethshj/Programs/Cr_data/13NOV24" 


### List the references that were downloaded from Full_polarization.py.
(Future task includes doing a check between this notebook and Full_polarization.py to ensure that the references are the same, rather
than hard coding it like this)

In [3]:
reference_cifs_mp_id = {
       "mp-644481", #Sc
       "mp-1215", #Ti
       "mp-18937", #V
       "mp-19177", #Cr
       "mp-510408", #Mn
       "mp-19770", #Fe
       "mp-22408", #Co
       "mp-19009", #Ni
       "mp-704645", #Cu
       "mp-2133" #Zn
    }

In [4]:
transition_metals = [
    "Sc", "Ti", "V", "Cr", "Mn", "Fe", "Co", "Ni", "Cu", "Zn"
]

def make_spectral_dictionary(parent_dir:str, reference_cifs_mp_id:list, elements_list:list):
    """
    We create a tuple of all of the files. We use a named tuple so we can access
    the individual indices by name ie metal_spectra_files[reference], rather than
    by number, ie metal_spectra_files[0]

    ARGS:
    parent_dir: A string that represents the entry point of the calculation directory
    created by the Full_polarization.py module

    reference_cifs_mp_id: A list of strings of mp-ids that you want to explicitly use as references.
    
    elements_list: A list of strings elements that you want to get a reference of. You usually want
    to choose an mp-id of an investigated element that is hooked up to an oxygen with a low
    Hull energy. Do not include the oxygen in your element list, just the complementary element.

    OUTPUT:
    metal_data: A dictionary whose keys and values are grouped in tuples.
    -'reference' : Path object to the reference cif that you want to subtract
    -'files' : all of the files with the respective _{atom} suffix  
    """

    element_spectra_files = namedtuple("element_spectra_files", ['reference', 'xes_files'])

    element_spectra_path_dictionary = {}

    # Loop through each transition metal and search for files that match the pattern
    for element in elements_list:
        print(f"Processing element {element}")

        reference_path = None
        for reference in reference_cifs_mp_id:

            print(f"Processing mp-id {reference}")

            reference_pattern = f"{parent_dir}/**/{reference}_{element}/Corvus.cfavg.xes.out"
            print(f"This is the reference pattern I am looking for {reference_pattern}")

            reference_file = glob.glob(reference_pattern, recursive=True)
            print(f"This is the files with the matching string I was able to find {reference_file}")

            if reference_file:
                reference_path = Path(reference_file[0])
                break
        
        if not reference_path:
            print(f"No reference file found for element {element}")

        # Use glob to search for files where the transition metal appears after the underscore
        xes_pattern = f"{parent_dir}/**/mp-**_{element}/Corvus.cfavg.xes.out"
        print(f"This is the xes pattern I am looking for {xes_pattern}")

        xes_files = glob.glob(xes_pattern, recursive=True)
        print(f"This is the xes pattern I was able to find {xes_files}")

        if not xes_files:
            reference_path = print(f"No xes files found for element {element}")

        xes_paths = [Path(xes_file) for xes_file in xes_files]
        print(f"This is the list of Path objects I was able to create {xes_paths}")

        element_spectra_path_dictionary[element] = element_spectra_files(
            reference = reference_path,
            xes_files = xes_paths
        )

    return element_spectra_path_dictionary
    
metal_data = make_spectral_dictionary(parent_dir, reference_cifs_mp_id, transition_metals) 

Processing element Sc
Processing mp-id mp-2133
This is the reference pattern I am looking for /home/sethshj/Programs/Cr_data/13NOV24/**/mp-2133_Sc/Corvus.cfavg.xes.out
This is the files with the matching string I was able to find []
Processing mp-id mp-510408
This is the reference pattern I am looking for /home/sethshj/Programs/Cr_data/13NOV24/**/mp-510408_Sc/Corvus.cfavg.xes.out
This is the files with the matching string I was able to find []
Processing mp-id mp-18937
This is the reference pattern I am looking for /home/sethshj/Programs/Cr_data/13NOV24/**/mp-18937_Sc/Corvus.cfavg.xes.out
This is the files with the matching string I was able to find []
Processing mp-id mp-19770
This is the reference pattern I am looking for /home/sethshj/Programs/Cr_data/13NOV24/**/mp-19770_Sc/Corvus.cfavg.xes.out
This is the files with the matching string I was able to find []
Processing mp-id mp-22408
This is the reference pattern I am looking for /home/sethshj/Programs/Cr_data/13NOV24/**/mp-22408_Sc

In [5]:

for element, data in metal_data.items():
    print(f"Element: {element}")
    print(f"    Reference: {data.reference}")
    print(f"    Matching Files: {data.xes_files}") 

print("Total number of matching files: ", sum(len(data.xes_files) for data in metal_data.values()))


Element: Sc
    Reference: /home/sethshj/Programs/Cr_data/13NOV24/mp-644481/mp-644481_Sc/Corvus.cfavg.xes.out
    Matching Files: [PosixPath('/home/sethshj/Programs/Cr_data/13NOV24/mp-18961/mp-18961_Sc/Corvus.cfavg.xes.out'), PosixPath('/home/sethshj/Programs/Cr_data/13NOV24/mp-644481/mp-644481_Sc/Corvus.cfavg.xes.out')]
Element: Ti
    Reference: /home/sethshj/Programs/Cr_data/13NOV24/mp-1215/mp-1215_Ti/Corvus.cfavg.xes.out
    Matching Files: [PosixPath('/home/sethshj/Programs/Cr_data/13NOV24/mp-1031430/mp-1031430_Ti/Corvus.cfavg.xes.out'), PosixPath('/home/sethshj/Programs/Cr_data/13NOV24/mp-1034067/mp-1034067_Ti/Corvus.cfavg.xes.out'), PosixPath('/home/sethshj/Programs/Cr_data/13NOV24/mp-1034272/mp-1034272_Ti/Corvus.cfavg.xes.out'), PosixPath('/home/sethshj/Programs/Cr_data/13NOV24/mp-1044311/mp-1044311_Ti/Corvus.cfavg.xes.out'), PosixPath('/home/sethshj/Programs/Cr_data/13NOV24/mp-1093769/mp-1093769_Ti/Corvus.cfavg.xes.out'), PosixPath('/home/sethshj/Programs/Cr_data/13NOV24/mp-10

# Delete linear backgrond using reference spectra

## Initial Examinations

### 1. Make the pandas dataframe

In [ ]:
def create_element_dataframes_with_parent_names(metal_data):
    """
    Creates DataFrames for the reference file and all XES files, with the parent directory names.

    Args:
        metal_data: Dictionary where keys are elements and values are named tuples
                    with 'reference' and 'xes_files'.

    Returns:
        None
    """
    for element, data in metal_data.items():
        # Read the reference data and assign it to reference_df
        reference_df = pd.read_csv(
            data.reference, sep=r'\s+', header=None, names=['Energy', 'x_polarization', 'y_polarization', 'z_polarization', 'Isotropic']
        )
        reference_df.set_index('Energy', inplace=True)
        print(f"\nElement: {element} - Reference Data")
        print(reference_df)

        # Loop through each XES file and create DataFrames
        for xes_file in data.xes_files:
            xes_df = pd.read_csv(
                xes_file, sep=r'\s+', header=None, names=['Energy', 'x_polarization', 'y_polarization', 'z_polarization', 'Isotropic']
            )
            xes_df.set_index('Energy', inplace=True)
            print(f"\nElement: {element} - XES Data for file {xes_file.parent.name}")
            print(xes_df)

create_element_dataframes_with_parent_names(metal_data)

In [18]:
def compare_reference_to_xes(metal_data):
    """
    Compare reference DataFrame to all respective XES DataFrames for every element.

    Args:
        metal_data: Dictionary where keys are elements and values are named tuples
                    with 'reference' and 'xes_files'.

    Returns:
        None
    """
    for element, data in metal_data.items():
        # Read reference data
        reference_df = pd.read_csv(
            data.reference, sep=r'\s+', header=None, names=['Energy', 'x_polarization', 'y_polarization', 'z_polarization', 'Isotropic']
        )
        #reference_df.set_index('Energy', inplace=True)
        reference_energy = reference_df['Energy']
        
        print(f"\nProcessing element: {element}")
        print(f"Reference file: {data.reference}")

        # Loop through each XES file and compare
        for xes_file in data.xes_files:
            # Read XES data
            xes_df = pd.read_csv(
                xes_file, sep=r'\s+', header=None, names=['Energy', 'x_polarization', 'y_polarization', 'z_polarization', 'Isotropic']
            )
            #xes_df.set_index('Energy', inplace=True)
            xes_energy = xes_df['Energy']

            if reference_df.shape == xes_df.shape:
                pass

                if not np.allclose(reference_energy.values, xes_energy.values, atol=1e-2):
                    print(f"Energy mismatch for XES file {xes_file.parent.name} of Element {element}. Aligning indices...")

                    # Calculate column-wise differences
                    energy_differences = xes_energy - reference_energy

                    # Compute the average of differences per column
                    avg_differences = energy_differences.mean()

                    print(f"Average differences for XES file {xes_file.parent.name}:")
                    print(avg_differences)
                
                else:
                    print(f"Energy columns match within tolerance for XES file {xes_file.parent.name} of Element {element}.")
            
            else:
                print(f"There is a shape mismatch between the reference {data.reference} and the xes file {xes_file.parent.name}")
                print(f"    reference: {reference_df.shape}")
                print(f"    xes file: {xes_df.shape}")

compare_reference_to_xes(metal_data)


Processing element: Sc
Reference file: /home/sethshj/Programs/Cr_data/13NOV24/mp-644481/mp-644481_Sc/Corvus.cfavg.xes.out
Energy mismatch for XES file mp-18961_Sc of Element Sc. Aligning indices...
Average differences for XES file mp-18961_Sc:
-1.836773991703932
Energy columns match within tolerance for XES file mp-644481_Sc of Element Sc.

Processing element: Ti
Reference file: /home/sethshj/Programs/Cr_data/13NOV24/mp-1215/mp-1215_Ti/Corvus.cfavg.xes.out
Energy mismatch for XES file mp-1031430_Ti of Element Ti. Aligning indices...
Average differences for XES file mp-1031430_Ti:
-2.070780412454912
Energy mismatch for XES file mp-1034067_Ti of Element Ti. Aligning indices...
Average differences for XES file mp-1034067_Ti:
-2.092550415855793
Energy mismatch for XES file mp-1034272_Ti of Element Ti. Aligning indices...
Average differences for XES file mp-1034272_Ti:
-2.0326846607549203
Energy mismatch for XES file mp-1044311_Ti of Element Ti. Aligning indices...
Average differences for 

### 1. Visually inspect the data

### Reference spectra to delete linear background

In [ ]:
for metal, data in metal_data.items():
    reference_file = data.reference
    if not Path(reference_file).exists():
        print(f"No matching reference file found for {metal}: {reference_file}")
        continue

    try:

        # Read reference data
        print(f"Processing reference file for {metal}: {reference_file}")
        reference_data = pd.read_csv(reference_file, sep=r'\s+', header=None, skipinitialspace=True)
        reference_data.columns = ['Energy', 'x_polarization', 'y_polarization', 'z_polarization', 'Isotropic']

        # Validate isotropic column
        for i, row in reference_data.iterrows():
            avg_value = (row['x_polarization'] + row['y_polarization'] + row['z_polarization']) / 3
            if not isclose(avg_value, row['Isotropic'], rel_tol=1e-5):
                raise ValueError(
                    f"Row {i}: Isotropic value does not match average polarization."
                )

        # Plot data
        plt.figure(figsize=(9, 6))
        plt.plot(reference_data['Energy'], reference_data['x_polarization'], label=f'{Path(reference_file).stem} x polarization reference')
        plt.plot(reference_data['Energy'], reference_data['y_polarization'], label=f'{Path(reference_file).stem} y polarization reference')
        plt.plot(reference_data['Energy'], reference_data['z_polarization'], label=f'{Path(reference_file).stem} z polarization reference')
        plt.plot(reference_data['Energy'], reference_data['Isotropic'], label=f'{Path(reference_file).stem} Isotropic reference')

        # Add plot settings
        plt.xlabel('Energy (eV)')
        plt.ylabel('Relative Intensity (arb. units)')
        plt.title(f'Reference Data for {metal}')
        plt.legend(loc='best')
        plt.grid(True)
        plt.show()

    except Exception as e:
        print(f"Error reading reference data for {metal}: {reference_file}: {e}")

### Calculated Spectra

In [ ]:
for metal, data in metal_data.items():
    if not data.files:
        print(f"No matching XES files found for: {metal}")
        continue

    #Read all of the other data
    for xes_file in data.files[:20]:
        try:
            # Read data
            print(f"Attempting to graph one of the XES files {xes_file}")
            data = pd.read_csv(xes_file, sep=r'\s+', header=None, skipinitialspace=True)
            data.columns = ['Energy', 'x_polarization', 'y_polarization', 'z_polarization', 'Isotropic']

            # Validate isotropic column
            for i, row in data.iterrows():
                avg_value = (row['x_polarization'] + row['y_polarization'] + row['z_polarization']) / 3
                if not isclose(avg_value, row['Isotropic'], rel_tol=1e-5):
                    raise ValueError(
                        f"Row {i}: Isotropic value does not match average polarization."
                    )

            # Plot data

            plt.figure(figsize=(9, 6))
            plt.plot(data['Energy'], data['x_polarization'], label=f'{Path(xes_file).parent.name} x polarization ')
            plt.plot(data['Energy'], data['y_polarization'], label=f'{Path(xes_file).parent.name} y polarization ')
            plt.plot(data['Energy'], data['z_polarization'], label=f'{Path(xes_file).parent.name} z polarization ')
            plt.plot(data['Energy'], data['Isotropic'], label=f'{Path(xes_file).parent.name} Isotropic ')

            # Add plot settings
            plt.xlabel('Energy (eV)')
            plt.ylabel('Relative Intensity (arb. units)')
            plt.title(f'XES Plot for {Path(xes_file).parent.name}')
            plt.legend(loc='best')
            plt.grid(True)
            plt.show()

        except Exception as e:
            print(f"Error reading {xes_file}: {e}")

    # Visualize .cif files
    # for cif_file in cif_files:
    #     try:
    #         print(f"Visualizing CIF file: {cif_file}")
    #         viewer = WeasWidget()
    #         viewer.load_structure(cif_file)
    #         display(viewer)
    #     except Exception as e:
    #         print(f"Error visualizing CIF file {cif_file}: {e}")


### Linear background subtraction

In [ ]:
# This is your standalone cell

# Check that 'metal_data' exists
if 'metal_data' not in globals():
    print("Error: 'metal_data' has not been defined. Please run the previous cells first.")
else:
    # Loop through each metal and process the files for each metal
    for metal, data in metal_data.items():
        if not data.xes_files:
            print(f"No matching XES files found for: {metal}")
            continue

        # Read the reference data for the current metal
        reference_file = data.reference
        if not Path(reference_file).exists():
            print(f"No matching reference file found for {metal}: {reference_file}")
            continue

        try:
            # Read reference data
            print(f"Processing reference file for {metal}: {reference_file}")
            reference_data = pd.read_csv(reference_file, sep=r'\s+', header=None, skipinitialspace=True)
            reference_data.columns = ['Energy', 'x_polarization', 'y_polarization', 'z_polarization', 'Isotropic']

            # Validate isotropic column
            for i, row in reference_data.iterrows():
                avg_value = (row['x_polarization'] + row['y_polarization'] + row['z_polarization']) / 3
                if not isclose(avg_value, row['Isotropic'], rel_tol=1e-5):
                    raise ValueError(
                        f"Row {i}: Isotropic value does not match average polarization."
                    )

        except Exception as e:
            print(f"Error reading reference data for {metal}: {reference_file}: {e}")
            continue

        # Process the XES files for the current metal
        for xes_file in data.xes_files:
            try:
                # Read XES data
                print(f"Attempting to graph one of the XES files {xes_file}")
                xes_data = pd.read_csv(xes_file, sep=r'\s+', header=None, skipinitialspace=True)
                xes_data.columns = ['Energy', 'x_polarization', 'y_polarization', 'z_polarization', 'Isotropic']

                # Validate isotropic column
                for i, row in xes_data.iterrows():
                    avg_value = (row['x_polarization'] + row['y_polarization'] + row['z_polarization']) / 3
                    if not isclose(avg_value, row['Isotropic'], rel_tol=1e-5):
                        raise ValueError(
                            f"Row {i}: Isotropic value does not match average polarization."
                        )

                # Ensure that the 'Energy' column is set as index for alignment
                xes_data.set_index('Energy', inplace=True)
                reference_data.set_index('Energy', inplace=True)

                # Directly subtract the reference data from the XES data for the specific metal
                diff_data = xes_data - reference_data[['x_polarization', 'y_polarization', 'z_polarization', 'Isotropic']]

                # Plot the differences
                plt.figure(figsize=(9, 6))
                plt.plot(diff_data.index, diff_data['x_polarization'], label=f'{Path(xes_file).parent.name} x polarization difference')
                plt.plot(diff_data.index, diff_data['y_polarization'], label=f'{Path(xes_file).parent.name} y polarization difference')
                plt.plot(diff_data.index, diff_data['z_polarization'], label=f'{Path(xes_file).parent.name} z polarization difference')
                plt.plot(diff_data.index, diff_data['Isotropic'], label=f'{Path(xes_file).parent.name} Isotropic difference')

                # Add plot settings
                plt.xlabel('Energy (eV)')
                plt.ylabel('Difference in Relative Intensity (arb. units)')
                plt.title(f'XES Plot vs Reference for {Path(xes_file).parent}')
                plt.legend(loc='best')
                plt.grid(True)
                plt.show()

            except Exception as e:
                print(f"Error reading {xes_file}: {e}")
    

### 2. Choose the best anisotropy and isotropy candidates for the job
##### (Open project: determine an algorithm to quantify anisotropy and give the user best possible candidates)

In [ ]:
###EXPECTED ANISOTROPY###

#input only the name of the material from the .cif file.
#ex: if you calculated FeO.cif, just input 'FeO' in the list.
anistropic_materials_name = ['', '']

anisotropic_matching_files = []
for aniso_material in anistropic_materials_name:
    for metal in transition_metals:
        
        # Use glob to search for files of the anisotropy material name, and the transition metal appears after the underscore
        pattern = f'{parent_dir}**/{aniso_material}_{metal}/Corvus.cfavg.xes.out'
        anistropic_materials_name.extend(glob.glob(pattern, recursive=True))
    
print(f'These are the matching_files list: {anisotropic_matching_files}')

###EXPECTED ISOTROPY###
isotropic_materials_name = ['', '']

isotropic_matching_files = []
for iso_material in isotropic_materials_name:
    for metal in transition_metals:
        
        # Use glob to search for files where the transition metal appears after the underscore
        pattern = f'{parent_dir}**/{iso_material}_{metal}/Corvus.cfavg.xes.out'
        isotropic_materials_name.extend(glob.glob(pattern, recursive=True))

print(f'These are the matching_files list: {isotropic_matching_files}')

### 3. Do some math and check if it makes sense

In [ ]:
def xes_integrated_abs_difference(data_1, data_2):
    ''' 
    Calculate the integrated absolute difference of Δμ(E) for data_1 and data_2.

    Data 1 and 2 have orthogonal polarizations.

    Parameters:
    data_1 (pandas.DataFrame): The DataFrame containing the X-ray absorption data for polarization 1.
    data_2 (pandas.DataFrame): The DataFrame containing the X-ray absorption data for polarization 2.

    Returns:
    difference (float): The integrated absolute difference of Δμ(E) between the two datasets.
    '''
    # Calculate the absolute difference of Delta mu(E)
    abs_difference = np.abs(data_1 - data_2)

    # Integrate the absolute difference over the energy range
    integrated_abs_difference = np.sum(abs_difference)

    return integrated_abs_difference

aniso_integrated_diff = xes_integrated_abs_difference()


In [ ]:
def xes_average(data_1, data_2, data_3):
    '''
    Calculates the integral of the average of 3 orthogonally polarized XES spectra.

    Parameters:
    data_1 (pandas.DataFrame): The DataFrame containing the X-ray absorption data for polarization 1.
    data_2 (pandas.DataFrame): The DataFrame containing the X-ray absorption data for polarization 2.
    data_3 (pandas.DataFrame): The DataFrame containing the X-ray absorption data for polarization 3.

    Return:
    integrated_average (float): The integral of the average of the 3 spectra.
    '''
    # Calculate the average of Delta mu(E) values
    average_delta_muE = (data_1 + data_2 + data_3) / 3

    # Integrate the average Delta mu(E) over the energy range
    integrated_average = np.sum(average_delta_muE)

    return integrated_average



In [ ]:
def anisotropy_parameter(xes_difference, xes_average):
    '''
    Calculate the anisotropy parameter, which is the quotient of the XES difference and the XES average.

    Parameters:
    xes_difference (float): The integrated absolute difference of Δμ(E).
    xes_average (float): The integral of the average of the 3 spectra.

    Returns:
    float: The anisotropy parameter.
    '''
    if xes_average == 0:
        raise ValueError(
            "The xes_average must not be zero to avoid division by zero.")

    return xes_difference / xes_average


In [ ]:

def anisotropy_matrix(data_x, data_y, data_z):
    '''
    Calculate a 3x3 anisotropy matrix where each entry represents the anisotropy parameter 
    for the difference between two datasets divided by the average of all three datasets.

    Parameters:
    data_x (pandas.DataFrame): The DataFrame containing the X-ray absorption data for polarization x.
    data_y (pandas.DataFrame): The DataFrame containing the X-ray absorption data for polarization y.
    data_z (pandas.DataFrame): The DataFrame containing the X-ray absorption data for polarization z.

    Returns:
    numpy.ndarray: A 3x3 anisotropy matrix.
    '''
    # Calculate the XES average of all three datasets
    xes_avg = xes_average(data_x, data_y, data_z)

    # Initialize a 3x3 matrix
    anisotropy_mat = np.zeros((3, 3))

    # Define the pairs for which to calculate the differences
    pairs = [(data_x, data_y), (data_x, data_z), (data_y, data_z)]

    # Fill the anisotropy matrix with the anisotropy parameters
    for i, (data1, data2) in enumerate(pairs):
        diff = xes_integrated_abs_difference(data1, data2)
        anisotropy_mat[i][(i+1) % 3] = anisotropy_parameter(diff, xes_avg)
        anisotropy_mat[(i+1) % 3][i] = anisotropy_mat[i][(i+1) %
                                                         3]  # Symmetric entries

    return anisotropy_mat

In [ ]:
def read_data(file):
    return np.loadtxt(file)

output_txt = f'{parent_dir}anisotropy_data.csv'

# Open the CSV file for writing
with open(output_txt, mode='w', newline='') as file_out:
    txt_writer = csv.writer(file_out, delimiter= '\t')
    
    # Write the header row
    txt_writer.writerow(['parent_dir', 'm00', 'm01', 'm02', 'm10', 'm11', 'm12', 'm20', 'm21', 'm22'])
    
    # Loop through each matching file and process the data
    for file in matching_files:
        data = read_data(file)
        data_x = data[:, 1]
        data_y = data[:, 2]
        data_z = data[:, 3]
        
        # Calculate the anisotropy matrix (3x3)
        anisotropy_mat = anisotropy_matrix(data_x, data_y, data_z)
        
        # Get the parent directory name
        parent_dir = os.path.basename(os.path.dirname(file))
        
        # Flatten the 3x3 matrix into a single row (list)
        flattened_matrix = anisotropy_mat.flatten().tolist()
        
        # Prepend the parent_dir to the flattened matrix row
        row = [parent_dir] + flattened_matrix
        
        # Write the row to the CSV
        txt_writer.writerow(row)

print(f"Data has been written to {output_txt}")
    

# (WIP) PROBABILITY DISTRIBUTION PARAMETER GUESSER
##### Perhaps some deconvolution data science can be in order?

In [ ]:
def LORENTZIAN_MODEL(center, num, sigma= 0.15, amp= 4000):
    """We know we want to set up lots of Lorentzians. Here is a function that sets one up fast.
    The function will stay in all caps unlike regular functions because I'm too lazy to change it after the
    first time I wrote this. This will be filled with data at the bottom of the cell"""

    model = LorentzianModel(prefix= num)
    modelparams = model.make_params()
    modelparams[num + 'center'].set(center)
    modelparams[num + 'amplitude'].set(amp)
    modelparams[num + 'sigma'].set(sigma, min=0)
    return model, modelparams

def central_limit_theorem_slope_change(independent_column, dependent_column)->list:
    """This takes two columns of data (Corvus.cfavg.out, XMU.dat, or any other spectral data in two columns)
    and conducts the central limit theorem. We store the zero points and put them in a list for our model guesser."""
    
    zipped_data = list(zip(independent_column, dependent_column))
    
    slope_changes = []
    
    for i in range(1, len(zipped_data) - 1):
        x1, y1 = zipped_data[i-1]  
        x2, y2 = zipped_data[i]    
        x3, y3 = zipped_data[i+1]
        
        slope1 = (y2 - y1) / (x2 - x1) if (x2 - x1) != 0 else 0
        slope2 = (y3 - y2) / (x3 - x2) if (x3 - x2) != 0 else 0
        

        if slope1 > 0 and slope2 < 0:
            slope_changes.append(x2)  # store the point where the change occurs. We only want the x (eV). If you want both poits, insert (x2,y2) instead
    
    return slope_changes

#BigModel= QuadraticModel(label= 'Background')
#AllParams= BigModel.make_params(a=0, b=0, c=0)

for file in matching_files:
    data = read_data(file)
    energy = data[:, 0]
    data_x = data[:, 1]
    data_y = data[:, 2]
    data_z = data[:, 3]
    data_isotropic = data[:, 4]

slope_changes_x_pol = central_limit_theorem_slope_change(data[:, 0],data[:, 1])
#slope_changes_y_pol = central_limit_theorem_slope_change(data[: 0],data[: 2])
#slope_changes_z_pol = central_limit_theorem_slope_change(data[: 0],data[: 3])
#slope_changes_isotropic = central_limit_theorem_slope_change(data[: 0],data[: 4])

print(slope_changes_x_pol)
AllParams = None
BigModel = []

for i, cen in enumerate(slope_changes_x_pol):
    model, modelparams = LORENTZIAN_MODEL(cen, 'data%d_' % (i+1) )
    AllParams = None
    BigModel = model
    AllParams.update(modelparams)

calculation = BigModel.eval(AllParams, x= data_x)
calculation_fit = BigModel.fit(data[:, 0], AllParams, x= data_x)
calculationVariables = calculation_fit.eval_components()

print(calculation_fit.fit_report(min_correl=0.5))

plt.plot(energy, data_x, label = 'X Polarization')
plt.plot(energy, calculation_fit.best_fit, label='best fit')
for name, comp in calculationVariables.items():
    plt.plot(data[:, 0], comp, '--', label=name)
plt.legend()
plt.title('Lines of best fit')
plt.xlabel('Energy (eV)')
plt.ylabel('Relative Intensity (arb. units)')
plt.show()
myfig = plt.figure(figsize=(12,9)) 